# DIA3. Transformación avanzada y enriquecimiento

## Ejercicio: Transformaciones avanzadas en dataset de e-commerce
---

### 1.- Base de datos

In [1]:
import pandas as pd
import numpy as np

# Clientes
clientes = pd.DataFrame({
    'cliente_id': range(1, 6),
    'nombre': ['Ana', 'Juan', 'María', 'Pedro', 'Laura'],
    'segmento': ['Premium', 'Regular', 'Premium', 'Regular', 'VIP']
})

# Pedidos
pedidos = pd.DataFrame({
    'pedido_id': range(1, 11),
    'cliente_id': np.random.choice(range(1, 6), 10),
    'producto': np.random.choice(['A', 'B', 'C', 'D'], 10),
    'precio': np.random.uniform(50, 500, 10).round(2),
    'fecha': pd.date_range('2024-01-01', periods=10)
})

print("Clientes y pedidos cargados")

ModuleNotFoundError: No module named 'pandas'

### 2.- Enriquecer datos con join

In [ ]:
# Unir pedidos con información de clientes
pedidos_enriquecidos = pd.merge(
    pedidos,
    clientes,
    on='cliente_id',
    how='left'
)

print("Pedidos con información de clientes:")
print(pedidos_enriquecidos.head())

Pedidos con información de clientes:
   pedido_id  cliente_id producto  precio      fecha nombre segmento
0          1           2        C  285.42 2024-01-01   Juan  Regular
1          2           3        D  441.50 2024-01-02  María  Premium
2          3           4        C  269.81 2024-01-03  Pedro  Regular
3          4           4        B  454.67 2024-01-04  Pedro  Regular
4          5           3        C  153.09 2024-01-05  María  Premium


### 3.- Calcular métricas por clientes

In [ ]:
# Calcular métricas por cliente
metricas_cliente = pedidos_enriquecidos.groupby(['cliente_id', 'nombre', 'segmento']).agg({
    'pedido_id': 'count',
    'precio': ['sum', 'mean', 'max'],
    'fecha': 'max'  # Última compra
}).round(2)

# Aplanar columnas multi-nivel
metricas_cliente.columns = ['num_pedidos', 'total_gastado', 'gasto_promedio', 'gasto_maximo', 'ultima_compra']
metricas_cliente = metricas_cliente.reset_index()

print("\nMétricas por cliente:")
print(metricas_cliente)


Métricas por cliente:
   cliente_id nombre segmento  num_pedidos  total_gastado  gasto_promedio  \
0           2   Juan  Regular            2         607.92          303.96   
1           3  María  Premium            2         594.59          297.30   
2           4  Pedro  Regular            3        1177.44          392.48   
3           5  Laura      VIP            3        1144.59          381.53   

   gasto_maximo ultima_compra  
0        322.50    2024-01-07  
1        441.50    2024-01-05  
2        454.67    2024-01-09  
3        485.53    2024-01-10  


### 4.- Validar reglas de negocios

In [ ]:
def validar_reglas_negocio(df):
    validaciones = []

    # VIP deben tener al menos 2 pedidos
    vip_insuficientes = df[(df['segmento'] == 'VIP') & (df['num_pedidos'] < 2)]
    if len(vip_insuficientes) > 0:
        validaciones.append(f"VIPs con pocos pedidos: {len(vip_insuficientes)}")

    # Premium no deben exceder gasto máximo
    premium_excesivos = df[(df['segmento'] == 'Premium') & (df['gasto_maximo'] > 800)]
    if len(premium_excesivos) > 0:
        validaciones.append(f"Premiums con gastos excesivos: {len(premium_excesivos)}")

    return validaciones

reglas_incumplidas = validar_reglas_negocio(metricas_cliente)
print(f"\nReglas de negocio incumplidas: {reglas_incumplidas}")


Reglas de negocio incumplidas: []
